In [1]:
# 데이터 변환
import os, json, shutil
from collections import defaultdict

def coco_to_yolo(
    coco_root="../datasets/coco",          # 원본 COCO 스타일 폴더
    out_root="../datasets/coco_yolo"       # YOLO 변환 결과 폴더
):
    # 원본 경로 가정:
    # coco_root/
    #   train/  val/
    #   annotations/train_annotations.json, val_annotations.json

    os.makedirs(out_root, exist_ok=True)
    for split in ["train", "val"]:
        os.makedirs(os.path.join(out_root, "images", split), exist_ok=True)
        os.makedirs(os.path.join(out_root, "labels", split), exist_ok=True)

        ann_path = os.path.join(coco_root, "annotations", f"{split}_annotations.json")
        with open(ann_path, "r", encoding="utf-8") as f:
            coco = json.load(f)

        # category_id -> 0..K-1 contiguous
        cats = sorted(coco["categories"], key=lambda x: x["id"])
        cat_id_to_cls = {c["id"]: i for i, c in enumerate(cats)}
        cls_to_name = {i: c["name"] for i, c in enumerate(cats)}

        # image_id -> (w,h,filename)
        images = {img["id"]: img for img in coco["images"]}

        # image_id -> list of ann
        img_to_anns = defaultdict(list)
        for ann in coco["annotations"]:
            # crowd 등 원치 않으면 필터 가능
            if ann.get("iscrowd", 0) == 1:
                continue
            img_to_anns[ann["image_id"]].append(ann)

        # 이미지 복사 + 라벨 txt 생성
        for img_id, img in images.items():
            w, h = img["width"], img["height"]
            file_name = img["file_name"]
            src_img = os.path.join(coco_root, split, file_name)
            dst_img = os.path.join(out_root, "images", split, file_name)

            # 이미지 복사
            os.makedirs(os.path.dirname(dst_img), exist_ok=True)
            shutil.copy2(src_img, dst_img)

            # 라벨 파일(txt)
            label_path = os.path.join(out_root, "labels", split, os.path.splitext(file_name)[0] + ".txt")

            lines = []
            for ann in img_to_anns.get(img_id, []):
                x, y, bw, bh = ann["bbox"]  # COCO: x,y,w,h (pixel)
                if bw <= 0 or bh <= 0:
                    continue

                cls = cat_id_to_cls[ann["category_id"]]

                # YOLO: x_center,y_center,w,h (normalized)
                x_c = (x + bw / 2) / w
                y_c = (y + bh / 2) / h
                bw_n = bw / w
                bh_n = bh / h

                # 범위 클램프(안전)
                x_c = min(max(x_c, 0.0), 1.0)
                y_c = min(max(y_c, 0.0), 1.0)
                bw_n = min(max(bw_n, 0.0), 1.0)
                bh_n = min(max(bh_n, 0.0), 1.0)

                lines.append(f"{cls} {x_c:.6f} {y_c:.6f} {bw_n:.6f} {bh_n:.6f}")

            with open(label_path, "w", encoding="utf-8") as f:
                f.write("\n".join(lines))

        # dataset.yaml (한 번만 만들면 되지만 split마다 동일하게 생성해도 무방)
        yaml_path = os.path.join(out_root, "dataset.yaml")
        if not os.path.exists(yaml_path):
            names_lines = "\n".join([f"  {i}: {cls_to_name[i]}" for i in sorted(cls_to_name)])
            yaml = f"""path: {out_root}
train: images/train
val: images/val

names:
{names_lines}
"""
            with open(yaml_path, "w", encoding="utf-8") as f:
                f.write(yaml)

    print("✅ COCO -> YOLO 변환 완료")
    print(f"  - dataset.yaml: {os.path.join(out_root, 'dataset.yaml')}")

coco_to_yolo()


✅ COCO -> YOLO 변환 완료
  - dataset.yaml: ../datasets/coco_yolo\dataset.yaml


In [3]:
import os
from ultralytics import YOLO

# ---------------------------
# 설정
# ---------------------------
DATA_YAML = "../datasets/coco_yolo/dataset.yaml"
MODEL_PT  = "yolov8n.pt"   # 시작 가중치(빠름). 성능 더 원하면 yolov8s.pt / yolov8m.pt
EPOCHS    = 120

PASS_AP50 = 0.75
STOP_AP50 = 0.80

pass_saved = False

# 학습 결과 폴더(자동 생성)
PROJECT = "runs_det"
NAME    = "exp_ap50_target"

# ---------------------------
# 콜백: epoch마다 AP50만 출력 + 조건 저장/중지
# ---------------------------
def on_fit_epoch_end(trainer):
    """
    trainer.metrics: epoch 종료 시 검증 지표가 들어옵니다.
    버전에 따라 키 이름이 조금 다를 수 있어, 안전하게 여러 후보를 확인합니다.
    목표: AP@0.50(=mAP50)만 가져오기
    """
    global pass_saved

    m = trainer.metrics  # dict-like

    # 여러 버전 대비: 후보 키들
    cand_keys = [
        "metrics/mAP50(B)", "metrics/mAP50", "mAP50(B)", "mAP50",
        "metrics/precision(B)",  # fallback용(의미 다름) - 가능하면 쓰지 않음
    ]

    ap50 = None
    for k in cand_keys:
        if k in m:
            ap50 = float(m[k])
            break

    # 그래도 못 찾으면: metrics dict 전체에서 mAP50 포함 키 탐색
    if ap50 is None:
        for k, v in m.items():
            if "mAP50" in str(k):
                ap50 = float(v)
                break

    if ap50 is None:
        # AP50를 못 읽었으면 조용히 넘어감(버전/환경 차이)
        return

    epoch = trainer.epoch + 1
    print(f"epoch={epoch:03d}  AP50={ap50:.4f}")

    # 0.75 통과 저장 (멈추지 않음)
    if (ap50 >= PASS_AP50) and (not pass_saved):
        # best.pt를 pass 파일로 복사 저장
        best_pt = os.path.join(trainer.save_dir, "weights", "best.pt")
        if os.path.exists(best_pt):
            dst = os.path.join(trainer.save_dir, "weights", "pass_ap50_0.75.pt")
            import shutil
            shutil.copy2(best_pt, dst)
        pass_saved = True

    # 0.80 도달 시 저장 + 즉시 종료
    if ap50 >= STOP_AP50:
        best_pt = os.path.join(trainer.save_dir, "weights", "best.pt")
        if os.path.exists(best_pt):
            dst = os.path.join(trainer.save_dir, "weights", "pass_ap50_0.80.pt")
            import shutil
            shutil.copy2(best_pt, dst)

        print("🎯 STOP(AP50>=0.80) reached -> saved & stop")
        trainer.stop = True   # ✅ 즉시 종료


# ---------------------------
# 학습 실행
# ---------------------------
model = YOLO(MODEL_PT)

model.add_callback("on_fit_epoch_end", on_fit_epoch_end)

model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    imgsz=640,
    batch=16,          # GPU 메모리 부족하면 8로
    device=0,          # GPU
    workers=0,         # 요청하신 조건
    project=PROJECT,
    name=NAME,
    verbose=False      # 기본 로그 줄이기 (AP50만 찍고 싶으니까)
)


Ultralytics 8.4.12  Python-3.9.21 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../datasets/coco_yolo/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=120, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=exp_ap50_target2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True,

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001AFFF09FE20>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

In [17]:
import numpy as np

def _to_scalar(x):
    """Ultralytics metric이 scalar/np.array/list 어떤 형태든 안전하게 float로 변환"""
    if x is None:
        return None
    # torch tensor
    if hasattr(x, "detach"):
        x = x.detach().cpu().numpy()
    # numpy / list
    x = np.asarray(x)
    # scalar면 그대로, 배열이면 평균
    return float(x.mean()) if x.size > 1 else float(x.item())

def pretty_print_yolo_val(metrics, target_map50=0.75):
    p       = _to_scalar(getattr(metrics.box, "p", None))
    r       = _to_scalar(getattr(metrics.box, "r", None))
    map50   = _to_scalar(getattr(metrics.box, "map50", None))
#    map5095 = _to_scalar(getattr(metrics.box, "map", None))

    status = "✅ PASS" if (map50 is not None and map50 >= target_map50) else "❌ FAIL"

    print("\n" + "="*52)
    print(f"{status}  (Target: mAP50 ≥ {target_map50:.2f})")
    print("-"*52)
    if p is not None:       print(f"Precision (P) : {p:.3f}")
    if r is not None:       print(f"Recall (R)    : {r:.3f}")
    if map50 is not None:   print(f"mAP50         : {map50:.3f} ({map50*100:.1f}%)")
#    if map5095 is not None: print(f"mAP50-95      : {map5095:.3f} ({map5095*100:.1f}%)")
    print("="*52 + "\n")

pretty_print_yolo_val(metrics, target_map50=0.75)


✅ PASS  (Target: mAP50 ≥ 0.75)
----------------------------------------------------
Precision (P) : 0.762
Recall (R)    : 0.798
mAP50         : 0.792 (79.2%)

